# Программирование для всех

*Алла Тамбовцева, НИУ ВШЭ*

## Парсинг HTML-файлов с помощью `BeautifulSoup`

Мы уже немного познакомились со структурой html-страниц, теперь попробуем выгрузить информацию из реально существующей страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main`:

In [2]:
main = "https://nplus1.ru/"

Сформируем запрос, отправим его и получим ответ с помощью функции `get()` из `requests`:

In [3]:
page = requests.get(main) 

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [18]:
page

<Response [200]>

Объект page имеет тип `Response` и скрыт от наших глаз. Однако при его вызове мы видим число 200 – это код результата, который означает, что страница благополучно загружена.

У объекта типа `Response` есть атрибут `.text`, в котором хранится исходный код страницы, который мы можем посмотреть, нажав *Ctrl+U* в Chrome:

In [ ]:
page.text

Результат выше – это обычная строка, тип string. Выполнять поиск по такой строке неудобно, поэтому преобразуем эту строку в объект типа `BeautifulSoup`, который позволяет выполнять поиск по тегам:

In [19]:
soup = BeautifulSoup(page.text)

In [20]:
type(soup)

bs4.BeautifulSoup

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [24]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

В коде выше мы использовали метод `.find_all()`, который выполняет поиск по заданному тэгу и возвращает список частей кода HTML с выбранным тэгом. Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.

Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [25]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [5]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li) 

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [ ]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [6]:
link0 = news[0]
print(link0)

Отправим запрос к этой странице с одной новостью и обработаем исходный код страницы:

In [ ]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

Попробуем найти заголовок новости. Изучив исходный код страницы, заметим, что он находится в разных местах страницы. Возьмем самый простой вариант – тэг `<title>`:

In [ ]:
# find() вместо find_all(), так как результат точно будет один
# незачем писать soup0.find_all("title")[0] для извлечения единственного элемента списка,
# если можно сразу вернуть этот единственный элемент

soup0.find("title")

Извлечем текст, который заключен внутри тэгов:

In [26]:
title = soup0.find("title").text
title

'Четвероногого робота научили стоять на воротах'

Найдем теперь имя автора. Заметим, что оно находится в тэге `<meta>`, причем не просто в `<meta>`, а с атрибутом `name`, равным `mediator_author`. Чтобы учесть это уточнение в поиске, подадим на вход `.find()` словарь с названием атрибута и его значением:

In [27]:
soup0.find("meta", {"name" : "mediator_author"})

<meta content="Григорий Копиев" name="mediator_author"/>

Осталось извлечь имя автора из `content` по аналогии с тем, как мы ранее извлекали ссылку из `href`. Используем метод `.get()`:

In [ ]:
author = soup0.find("meta", {"name" : "mediator_author"}).get("content") 

Аналогичным образом извлечем дату публикации новости:

In [7]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 

С остальными характеристиками – сложностью и рубриками – все более интересно. С одной стороны, их можно найти точно так же по тэгам и атрибутам, но для надежности (не все названия атрибутов здесь понятны и гарантируют уникальность) давайте сузим пространство для поиска и будем искать их не по всей странице, а в пределах какого-нибудь раздела. Такой раздел есть, он имеет очень длинное значение атрибута `class`:

In [28]:
div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
div

<div class="flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9">
<span class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center">
<span class="group-hover:text-main transition-colors duration-75">16:04</span>
</span>
<a class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center" href="/news/2022/10/17">
<span class="group-hover:text-main transition-colors duration-75">17.10.22</span>
</a>
<a class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center" href="/material/difficulty/2">
<svg class="w-4 h-4 mr-1 group-hover:text-main transition-colors duration-75 stroke-current">
<use xlink:href="#n1_star"></use>
</svg>
<span class="group-hover:text-main transition-colors duration-75">2.4</span>
</a>
<a class="relative before:block before:w-px before:b

В этом разделе встречаются более мелкие части с тэгами `span`, в которых хранится дата, сложность статьи и рубрики. К сожалению, сайт сейчас в процессе перехода к новой структуре и новому дизайну, поэтому не всегда последовательность элементов будет такой (где-то будет вклиниваться время публикации статьи), но каким-то быстрым способом мы с этой проблемой сейчас не справимся. Поэтому отберем элементы как есть:

In [29]:
diffc = div.find_all("span")[1]
rubs_raw = div.find_all("span")[2:]

In [ ]:
print(diffc)
print(rubs_raw)

Теперь давайте извлечем из блоков кода HTML с рубриками текст с названиями рубрик:

In [30]:
rubs = [r.text for r in rubs_raw]
rubs

['17.10.22', '2.4', 'Роботы и дроны']

Склеим их в одну строку по запятой:

In [10]:
rubs_text = ", ".join(rubs)

Осталось аналогичным образом сохранить самое главное – текст новости. Заметим, что его нужно собирать по абзацам – по тэгам `<p>` с атрибутом `class`, равным `mb-6`:

In [31]:
pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars_raw

[<p class="text-36 md:text-44 xl:text-54 font-spectral text-main-gray mb-6">Его научили трем движениям для успешного перехвата</p>,
 <p class="mb-6">Американские инженеры научили четвероногого робота работать футбольным вратарем. Он анализирует траекторию полета мяча после удара и прыгает, чтобы перехватить его и не дать попасть в ворота. Статья о разработке <a href="https://arxiv.org/abs/2210.04435" rel="noreferrer noopener" target="_blank">опубликована</a> на arXiv.org.</p>,
 <p class="mb-6">Как правило, инженеры-робототехники решают прикладные задачи, которые можно напрямую применить на практике. Например, это создание эффективных алгоритмов и приспособлений для перемещения ходячих роботов по пересеченной местности. Но есть и немало работ, в которых инженеры, казалось бы, занимаются не особо полезными вещами, к примеру, <a href="https://nplus1.ru/news/2019/11/19/cassie-juggling" rel="noreferrer noopener" target="_blank">обучением</a> двуногого робота чеканке мячика. Однако в процесс

Извлечем текст в чистом виде из каждого элемента списка `pars_raw`:

In [11]:
pars = [p.text for p in pars_raw] 

Склеим полученные строки в одну большую строку через `.join()`, а заодно избавимся от посторонних символов `\xa0`, которые соответствуют неразрывным пробелам, и `\n`, которые соответствуют переходам на новую строку:

In [12]:
text = " ".join(pars)
text = text.replace("\xa0", " ") 
text = text.replace("\n", " ") 

Теперь все красиво. Напишем функцию для выгрузки информации по одной новости и применим ее к новостям на главной странице.

In [13]:
def get_news(link0):
    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text) 
    title = soup0.find("title").text
    author = soup0.find("meta", {"name" : "mediator_author"}).get("content") 
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[1].text
    rubs_raw = div.find_all("span")[2:]
    rubs = [r.text for r in rubs_raw]
    rubs_text = ", ".join(rubs)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ") 
    return title, author, date, diffc, rubs_text, text

Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [14]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [15]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.5)

https://nplus1.ru/news/2022/10/17/cetveronogogo-robota-naucili-otbivat-myaci
https://nplus1.ru/news/2022/10/17/mola-alexandrini
https://nplus1.ru/news/2022/10/17/cetveronogogo-robota-naucili-otbivat-myaci
https://nplus1.ru/news/2022/10/17/greenland
https://nplus1.ru/news/2022/10/17/gold-label
https://nplus1.ru/news/2022/10/17/tagar-rock-art
https://nplus1.ru/news/2022/10/15/barium
https://nplus1.ru/news/2022/10/15/scythians
https://nplus1.ru/news/2022/10/15/chinese-coins
https://nplus1.ru/news/2022/10/14/anguilla-anguilla
https://nplus1.ru/news/2022/10/14/oldest-grave
https://nplus1.ru/news/2022/10/14/brain-model-rats
https://nplus1.ru/news/2022/10/14/gorn
https://nplus1.ru/news/2022/10/14/polyarizuemost-protona
https://nplus1.ru/news/2022/10/14/direct-imaging-technology
https://nplus1.ru/news/2022/10/14/charge-transfer-photocatalysis
https://nplus1.ru/news/2022/10/14/pangur-ban
https://nplus1.ru/news/2022/10/14/charles-xii
https://nplus1.ru/news/2022/10/13/edmontosaurus-mummy
https://

Ура! Несмотря на то, что ближе к концу списка со ссылками мы стали получать сообщения об ошибках, новости за последние несколько дней сгрузились корректно (сайт в разработке, пока небольшие проблемы с унификацией и упорядочиванием ссылок).

Посмотрим на несколько элементов `info`:

In [33]:
info[10:12]

[('Археологи обнаружили древнейшее захоронение на севере Германии',
  'Михаил Подрезов',
  '2022-10-14',
  '2.3',
  'Археология, Антропология',
  'Ему около 10,5 тысячи лет Археологи провели новые раскопки на осушенном болоте Дювензее и обнаружили древнейшее захоронение на севере Германии. Оно представляет собой кремацию, совершенную, по предварительным оценкам, около 10,5 тысячи лет, то есть в эпоху раннего мезолита. Об этом сообщило Управление археологии федеральной земли Шлезвиг-Гольштейн на своем сайте. На территории современной федеральной земли Шлезвиг-Гольштейн находится осушенное торфяное болото Дювензее, известное благодаря археологическим находкам. Изначально Дювензее представляло собой озеро, образовавшееся на месте отступившего ледника и достигшее максимума своей полноводности в самом начале эпохи голоцена – в раннем пребореальном периоде. Но затем оно постепенно заилилось, превратившись в конечном счете в торфяное болото. Как археологический объект Дювензее стало известно 

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [16]:
import pandas as pd

In [34]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Четвероногого робота научили стоять на воротах,Григорий Копиев,2022-10-17,16:04,"17.10.22, 2.4, Роботы и дроны",Его научили трем движениям для успешного перех...
1,Короткую луну-рыбу весом 2744 килограмма призн...,Сергей Коленов,2022-10-17,17:20,"17.10.22, 1.7, Зоология",Она тяжелее предыдущего рекордсмена почти на п...
2,Четвероногого робота научили стоять на воротах,Григорий Копиев,2022-10-17,16:04,"17.10.22, 2.4, Роботы и дроны",Его научили трем движениям для успешного перех...
3,На гренландских археологических памятниках наш...,Михаил Подрезов,2022-10-17,14:50,"17.10.22, 3.3, Археология",Ученые исследовали 2500 фрагментов костей живо...
4,Плазмонный резонанс поможет определить зрелост...,Марат Хамадеев,2022-10-17,12:43,"17.10.22, 4.5, Химия, Физика",Для этого нужно запустить в напитке химическую...


Не везде информация сгрузилась корректно, есть проблемы унификации, можно потом поправить это, написав функцию и применив ее через `.apply()` к соответствующим столбцам.

А пока добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [36]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]
df.to_excel("nplus1.xlsx")